In [1]:
import sys, os
import numpy as np
from math import acos
import math
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [2]:
def load_data(path):
    X=[]
    Y=[]
    # 101 faces - sub folders, 6 emotions - Angry, Disgust, Fear, Happy, Sad, Surprise
    for face_dir in os.listdir(path):
        face_path = path+face_dir+'/'
        if os.path.isdir(face_path):
            for label in os.listdir(face_path):
                emotion_path = face_path+label+'/'
                for file in os.listdir(emotion_path):
                    if file.endswith(".bnd") or file.endswith(".landmark"):
                        file_path = emotion_path+file
                        points = np.loadtxt(emotion_path+file, usecols=(1, 2, 3), encoding='utf-8') #x,y,z
                        X.append(points)
                        Y.append(label)
    X = np.array(X)
    Y = np.array(Y)
    X = X.reshape(X.shape[0], -1)    
    return X,Y  

def translate_data(X):
    X_translated = []
    for face in X:
        mean = np.mean(face)
        translated_face = face - mean
        X_translated.append(translated_face)
    X_translated = np.array(X_translated)
    return X_translated

def rotate_data(X, axis):
    pi=round(2*acos(0.0), 3)
    sin_angle = math.sin(pi) #approx 0
    cos_angle = math.cos(pi) #approx -1
    
    rotated_X=[]
    if axis == 'x':
        rotation_matrix = np.array([[1, 0, 0], [0, cos_angle, sin_angle], [0, -sin_angle, cos_angle ]])
    elif axis == 'y':
        rotation_matrix = np.array([[cos_angle, 0, -sin_angle], [0, 1, 0], [sin_angle, 0, cos_angle]])
    elif axis == 'z':
        rotation_matrix = np.array([[cos_angle, sin_angle, 0], [-sin_angle, cos_angle, 0], [0, 0, 1]])
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")
    for face in X:
        rotated_data = np.dot(face.reshape(-1, 3), rotation_matrix.T).reshape(face.shape)
        rotated_X.append(rotated_data)
    rotated_X = np.array(rotated_X) 
    return rotated_X

In [3]:
from sklearn.svm import SVC
def classification(X,Y, classifier_type='RF'):
    if classifier_type == 'RF':
        clf = RandomForestClassifier()
    elif classifier_type == 'SVM':
        clf = SVC()
    elif classifier_type == 'TREE':
        clf = DecisionTreeClassifier()

    Y_pred = []
    test_indices = []
    cv = KFold(n_splits=10)
    for (train,test) in cv.split(X,Y):
        clf.fit(X[train],Y[train])
        Y_pred.append(clf.predict(X[test]))
        test_indices.append(test)
    
    return Y,Y_pred,test_indices

def model_evaluation(pred, indices, y):
    finalPredictions = []
    groundTruth = []
    for p in pred:
        finalPredictions.extend(p)
    for i in indices:
        groundTruth.extend(y[i])
    print(confusion_matrix(finalPredictions, groundTruth))
    print("Precision: ", precision_score(groundTruth, finalPredictions, average='macro'))
    print("Recall: ", recall_score(groundTruth, finalPredictions, average='macro'))
    print("Accuracy: " , accuracy_score(groundTruth, finalPredictions))


In [4]:
path = './BU4DFE_BND_V1.1/'
#path = args[3]
X,Y = load_data(path)
X_1,Y_1 = X,Y


In [5]:
classifier = 'SVM'
data_type = 'o'
y_original, y_pred, indices = classification(X,Y, classifier_type=classifier)
print("Following are the evaluation metrics for", classifier,"classifier and", data_type, "data type")
#Performing Evaluation
model_evaluation(y_pred, indices, y_original)

Classifier - Support Vector Machine


In [ ]:
classifier = 'SVM'
data_type = 'x'
X = rotate_data(X_1,'x')
#X = translate_data(X)
y_original, y_pred, indices = classification(X,Y, classifier_type=classifier)
print("Following are the evaluation metrics for", classifier,"classifier and", data_type, "data type")
#Performing Evaluation
model_evaluation(y_pred, indices, y_original)

Following are the evaluation metrics for TREE classifier and x data type
[[3600 1593 1527 1462 1979  915]
 [1601 2667 1868 1241 1680 1197]
 [1355 1346 2074 1415 1361 1516]
 [ 687 1973 1529 3724 1765  884]
 [1727  941 1360 1042 2108  903]
 [1154 1651 1686 1089 1249 4643]]
Precision:  0.30533839783987543
Recall:  0.3111962599365155
Accuracy:  0.31094658910629297


In [ ]:
classifier = 'SVM'
data_type = 'y'
X = rotate_data(X_1,'y')
#X = translate_data(X)
y_original, y_pred, indices = classification(X,Y, classifier_type=classifier)
print("Following are the evaluation metrics for", classifier,"classifier and", data_type, "data type")
#Performing Evaluation
model_evaluation(y_pred, indices, y_original)

Following are the evaluation metrics for TREE classifier and y data type
[[3358 1795 1736 1816 1727 1059]
 [1732 2412 1667 1557 1470 1318]
 [1493 1372 1865 1476 1321 1155]
 [ 709 1704 1497 3477 2021  819]
 [1448  885 1263  560 2101  650]
 [1384 2003 2016 1087 1502 5057]]
Precision:  0.2960403113886501
Recall:  0.30218307253336657
Accuracy:  0.30192358540454783


In [ ]:
classifier = 'SVM'
data_type = 'z'
X = rotate_data(X_1,'z')
#X = translate_data(X)
y_original, y_pred, indices = classification(X,Y, classifier_type=classifier)
print("Following are the evaluation metrics for", classifier,"classifier and", data_type, "data type")
#Performing Evaluation
model_evaluation(y_pred, indices, y_original)

Following are the evaluation metrics for TREE classifier and z data type
[[3409 1854 1412 1608 1565  964]
 [1635 2247 1713 1522 1395 1174]
 [1398 1400 1946 1324 1521 1101]
 [ 901 1863 1511 3359 1774  810]
 [1422 1199 1511  811 2345  827]
 [1359 1608 1951 1349 1542 5182]]
Precision:  0.2979689785897864
Recall:  0.3057720392003615
Accuracy:  0.30552617662612375


In [ ]:
classifier = 'SVM'
data_type = 't'
X = translate_data(X_1)
y_original, y_pred, indices = classification(X,Y, classifier_type=classifier)
print("Following are the evaluation metrics for", classifier,"classifier and", data_type, "data type")
#Performing Evaluation
model_evaluation(y_pred, indices, y_original)

Following are the evaluation metrics for TREE classifier and t data type
[[2728 1463 1275 1345 1947  826]
 [2082 2648 1835 1629 1707 1133]
 [1305 1510 2567 1576 1449 1546]
 [1256 1370 1442 2979  846  889]
 [1279 1325  740  720 2430  835]
 [1474 1855 2185 1724 1763 4829]]
Precision:  0.30041389445197786
Recall:  0.3006336299030599
Accuracy:  0.3004528027498678
